In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("hf://datasets/maharshipandya/spotify-tracks-dataset/dataset.csv")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
#Checking loading of data frame
df.head(5)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [4]:
# Data Cleaning Function (modify)
def clean_data(df):
  clean_df = df.copy()
  clean_df = clean_df.drop('Unnamed: 0', axis=1, errors='ignore')
  clean_df = clean_df.dropna()

  #Encode explicit column (binary encoding)
  clean_df['explicit'] = clean_df['explicit'].astype(int)

  # Drop track_id, artisits,album_name
  clean_df = clean_df.drop(['track_id', 'artists', 'album_name','track_name'], axis=1)

  # Lets look at classification of track genres
  # Need to encode the track_genre
  le = LabelEncoder()
  clean_df['track_genre'] = le.fit_transform(clean_df['track_genre'])

  return clean_df

# Apply the function to the DataFrame
spotify_clean = clean_data(df)

In [5]:
#Looking at cleaned data frame
spotify_clean.sample(15)

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
68570,0,185386,0,0.816,0.776,8,-4.405,1,0.0562,0.23800,0.000004,0.0642,0.7040,94.996,4,68
39024,0,181386,0,0.555,0.526,9,-13.588,1,0.0422,0.87600,0.902000,0.2180,0.8900,90.608,3,39
76622,21,323000,0,0.132,0.424,0,-8.962,1,0.0341,0.78000,0.884000,0.0566,0.0653,78.928,4,76
46625,57,351026,0,0.705,0.866,8,-5.036,0,0.0563,0.17100,0.000162,0.6450,0.8160,127.986,4,46
76580,23,281133,0,0.567,0.655,7,-5.271,1,0.0334,0.16500,0.000622,0.1390,0.1150,96.007,4,76
53774,72,190487,0,0.677,0.766,6,-6.896,1,0.0568,0.02190,0.000007,0.1290,0.1980,123.062,4,53
18033,30,166106,0,0.336,0.397,9,-8.717,0,0.0606,0.69300,0.000000,0.8860,0.3550,174.686,3,18
62530,39,277044,0,0.437,0.961,11,-3.968,0,0.2310,0.08270,0.000041,0.3270,0.4740,147.930,4,62
54759,10,314714,0,0.403,0.788,1,-11.464,0,0.0642,0.35900,0.835000,0.1290,0.1310,104.407,4,54
74967,38,130037,0,0.521,0.461,9,-10.949,1,0.0768,0.67600,0.001700,0.0605,0.7050,180.341,3,74


In [6]:
#Standardize features (except last column)
scaler = StandardScaler()
# Explicitly cast the relevant columns to numeric types before scaling
numeric_cols = spotify_clean.select_dtypes(include=np.number).columns
spotify_clean[numeric_cols[:-1]] = spotify_clean[numeric_cols[:-1]].astype(float) # or another appropriate numeric type
spotify_clean.iloc[:, :-1] = scaler.fit_transform(spotify_clean.iloc[:, :-1])

## Application of Neural Network (NN) for classification

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [8]:
#Preparing data
X = spotify_clean.drop('track_genre', axis=1)
y = spotify_clean['track_genre']
y = to_categorical(y)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [14]:
# Build and Compile the Neural Network
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Model parameters
input_size = X_train.shape[1]
hidden_size = 64
output_size = len(set(y_train.argmax(axis=1)))  # Number of classes

# Initialize the model, loss function, and optimizer
model = NeuralNetwork(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert data to PyTorch tensors and create DataLoaders
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.argmax(axis=1), dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.argmax(axis=1), dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [15]:
# Early Stopping
# Parameters for early stopping
patience = 5
best_val_loss = float('inf')
counter = 0
checkpoint_path = 'checkpoint.pth'
early_stop = False


In [18]:
# Training the NN
num_epochs = 16
for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation phase
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
    val_loss /= len(test_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Validation Loss: {val_loss:.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), checkpoint_path)  # Save the best model
        print(f"Validation loss improved to {val_loss:.4f}. Saving model.")
    else:
        counter += 1
        print(f"No improvement. Early stopping counter: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered.")
            early_stop = True
            break

# Load the best model after early stopping
if early_stop:
    model.load_state_dict(torch.load(checkpoint_path))
    print("Model restored to best validation loss.")



Epoch 1/16, Train Loss: 2.7374, Validation Loss: 2.7875
Validation loss improved to 2.7875. Saving model.
Epoch 2/16, Train Loss: 2.7293, Validation Loss: 2.7773
Validation loss improved to 2.7773. Saving model.
Epoch 3/16, Train Loss: 2.7225, Validation Loss: 2.7771
Validation loss improved to 2.7771. Saving model.
Epoch 4/16, Train Loss: 2.7156, Validation Loss: 2.7673
Validation loss improved to 2.7673. Saving model.
Epoch 5/16, Train Loss: 2.7080, Validation Loss: 2.7649
Validation loss improved to 2.7649. Saving model.
Epoch 6/16, Train Loss: 2.7022, Validation Loss: 2.7610
Validation loss improved to 2.7610. Saving model.
Epoch 7/16, Train Loss: 2.6957, Validation Loss: 2.7605
Validation loss improved to 2.7605. Saving model.
Epoch 8/16, Train Loss: 2.6908, Validation Loss: 2.7471
Validation loss improved to 2.7471. Saving model.
Epoch 9/16, Train Loss: 2.6842, Validation Loss: 2.7480
No improvement. Early stopping counter: 1/5
Epoch 10/16, Train Loss: 2.6793, Validation Loss: 2.

In [19]:
# Print training
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(y_batch.tolist())
        y_pred.extend(predicted.tolist())

print("Classification Report:")
print(classification_report(y_true, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.18      0.11      0.14       213
           1       0.36      0.30      0.33       203
           2       0.19      0.02      0.04       215
           3       0.09      0.16      0.11       184
           4       0.30      0.30      0.30       197
           5       0.17      0.09      0.12       193
           6       0.45      0.65      0.53       210
           7       0.41      0.47      0.43       205
           8       0.07      0.02      0.04       214
           9       0.17      0.26      0.21       197
          10       0.46      0.36      0.40       199
          11       0.08      0.01      0.02       214
          12       0.18      0.20      0.19       193
          13       0.52      0.40      0.45       206
          14       0.37      0.37      0.37       214
          15       0.16      0.25      0.20       198
          16       0.49      0.44      0.46       198
    

## NN Training: Detail how you trained your NN including how you learned hyperparameters such as learning rate.

## Explain the metrics you are using to assess the performance of your NN
1. Accuracy:  Proportion of correctly classified genres
2. Precision, Recall, F1-Score: Evaluated per genre using a classification report to measure how well the NN captures each genre’s characteristics.
3. Confusion Matrix: Visualize class-wise prediction performance.
4. Loss Curves:Monitor training and validation loss to detect overfitting.
